<a href="https://colab.research.google.com/github/yourusername/custom-search-engine/blob/main/backend/training_tfidf_colab_for_antique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enhanced TF-IDF Training for Antique Dataset

This notebook trains TF-IDF models on the Antique dataset with **EXACT alignment** to the enhanced TF-IDF service.

## Key Features:
- **Exact text cleaning alignment** with `enhanced_tfidf_service.py` and `tfidf_text_cleaning_service.py`
- **Enhanced vectorizer parameters** (100k features, trigrams, etc.)
- **LSA semantic similarity** for reranking
- **Query expansion** using term co-occurrence
- **Complete evaluation** with MAP, MRR, Precision@10, Recall@10
- **Model compatibility** with the production enhanced TF-IDF service

## Installation and Setup

In [ ]:
# Install required packages
!pip install -q ir-datasets scikit-learn numpy joblib nltk tqdm symspellpy textblob matplotlib seaborn

In [ ]:
import ir_datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from tqdm import tqdm
import json
import os
import time
from typing import List, Dict, Any, Optional, Tuple, Set
from collections import defaultdict, Counter
import math

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully")
print("🎯 This notebook creates models EXACTLY compatible with enhanced_tfidf_service.py")

## Text Cleaning Service - EXACT Alignment

This implements the **EXACT SAME** text cleaning pipeline as `tfidf_text_cleaning_service.py` and `enhanced_tfidf_service.py`

In [ ]:
class TFIDFTextCleaner:
    """
    EXACT replica of TFIDFTextCleaningService from tfidf_text_cleaning_service.py
    This ensures 100% alignment with the enhanced TF-IDF service
    """
    
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        try:
            self.stop_words = set(stopwords.words('english'))
            print(f"✅ Loaded {len(self.stop_words)} English stopwords for TF-IDF")
        except Exception as e:
            print(f"⚠️ Could not load stopwords: {e}")
            self.stop_words = set()
    
    def clean_text_for_tfidf(self, text: str, preserve_document_structure: bool = True) -> Dict[str, Any]:
        """
        Clean text specifically for TF-IDF vectorization
        This implements the EXACT same steps as tfidf_text_cleaning_service.py
        """
        if not text or not text.strip():
            return {
                "original_text": text or "",
                "cleaned_text": "",
                "tokens": [],
                "token_count": 0,
                "processing_stats": {"empty_input": True}
            }
        
        original_text = text
        processing_stats = {
            "original_length": len(text),
            "steps_applied": []
        }
        
        # Step 1: Convert to lowercase (same as tfidf_text_cleaning_service.py)
        text = text.lower()
        processing_stats["steps_applied"].append("lowercase_conversion")
        
        # Step 2: Remove HTML tags (same as tfidf_text_cleaning_service.py)
        text = re.sub(r'<[^>]+>', '', text)
        processing_stats["steps_applied"].append("html_tag_removal")
        
        # Step 3: Clean special characters (same pattern as tfidf_text_cleaning_service.py)
        # Keep only alphanumeric characters and spaces
        text = re.sub(r'[^a-z0-9\s]', ' ', text)
        processing_stats["steps_applied"].append("special_character_removal")
        
        # Step 4: Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        processing_stats["steps_applied"].append("whitespace_normalization")
        
        # Step 5: Tokenization (same as tfidf_text_cleaning_service.py)
        tokens = word_tokenize(text)
        processing_stats["tokens_after_tokenization"] = len(tokens)
        processing_stats["steps_applied"].append("tokenization")
        
        # Step 6: Token filtering (same criteria as tfidf_text_cleaning_service.py)
        filtered_tokens = []
        for token in tokens:
            # Skip tokens that are too short or not alphanumeric
            if len(token) < 2 or not token.isalnum():
                continue
            
            # Skip stopwords (same as tfidf_text_cleaning_service.py)
            if token in self.stop_words:
                continue
                
            filtered_tokens.append(token)
        
        processing_stats["tokens_after_filtering"] = len(filtered_tokens)
        processing_stats["stopwords_removed"] = len(tokens) - len(filtered_tokens) - (len(tokens) - len([t for t in tokens if len(t) >= 2 and t.isalnum()]))
        processing_stats["steps_applied"].append("token_filtering")
        
        # Step 7: Lemmatization THEN Stemming (EXACT same order as tfidf_text_cleaning_service.py)
        final_tokens = []
        for token in filtered_tokens:
            # First lemmatize
            lemmatized = self.lemmatizer.lemmatize(token)
            # Then stem the lemmatized form
            stemmed = self.stemmer.stem(lemmatized)
            final_tokens.append(stemmed)
        
        processing_stats["steps_applied"].extend(["lemmatization", "stemming"])
        
        # Step 8: Create final cleaned text
        cleaned_text = " ".join(final_tokens)
        
        # Final processing statistics
        processing_stats.update({
            "final_token_count": len(final_tokens),
            "final_text_length": len(cleaned_text),
            "compression_ratio": len(cleaned_text) / len(original_text) if len(original_text) > 0 else 0,
            "stemmer_used": self.stemmer is not None,
            "lemmatizer_used": self.lemmatizer is not None,
            "stopwords_count": len(self.stop_words)
        })
        
        return {
            "original_text": original_text,
            "cleaned_text": cleaned_text,
            "tokens": final_tokens,
            "token_count": len(final_tokens),
            "processing_stats": processing_stats
        }
    
    def clean_text(self, text: str) -> str:
        """Simple interface that returns just the cleaned text"""
        result = self.clean_text_for_tfidf(text)
        return result["cleaned_text"]

# Initialize the text cleaner
text_cleaner = TFIDFTextCleaner()
print("✅ TF-IDF Text Cleaner initialized with EXACT alignment to enhanced_tfidf_service.py")

## Enhanced Vectorizer Parameters

These are the **EXACT SAME** parameters used in `enhanced_tfidf_service.py`

In [ ]:
def get_enhanced_vectorizer_params() -> Dict[str, Any]:
    """
    Get optimized TF-IDF parameters for higher MAP scores
    EXACT copy from enhanced_tfidf_service.py._get_enhanced_vectorizer_params()
    """
    return {
        'max_features': 100000,  # Increased from 10k to 100k
        'ngram_range': (1, 3),   # Include trigrams for better phrase matching
        'min_df': 2,             # Keep minimum document frequency low
        'max_df': 0.85,          # Slightly more restrictive for common terms
        'sublinear_tf': True,    # Apply log normalization to TF
        'norm': 'l2',            # L2 normalization
        'use_idf': True,         # Use IDF weighting
        'smooth_idf': True,      # Smooth IDF weights
        'token_pattern': r'(?u)\b\w\w+\b',  # Default pattern for word boundaries
        'strip_accents': 'unicode',  # Remove accents for better matching
    }

# Get the enhanced parameters
enhanced_params = get_enhanced_vectorizer_params()
print("Enhanced TF-IDF Parameters:")
for key, value in enhanced_params.items():
    print(f"  {key}: {value}")

print("\n✅ Using EXACT same parameters as enhanced_tfidf_service.py")

## Dataset Loading and Preprocessing

In [ ]:
def load_antique_dataset_with_enhanced_preprocessing():
    """
    Load Antique dataset with enhanced preprocessing matching the service
    """
    print("📚 Loading Antique dataset with enhanced preprocessing...")
    dataset = ir_datasets.load('antique/train')
    
    documents = []
    doc_metadata = []
    cleaned_texts = []
    
    # Load documents with enhanced preprocessing
    for doc in tqdm(dataset.docs_iter(), desc="Loading and cleaning documents"):
        # Clean text using the EXACT same method as enhanced_tfidf_service
        cleaned_result = text_cleaner.clean_text_for_tfidf(doc.text, preserve_document_structure=True)
        cleaned_text = cleaned_result["cleaned_text"]
        
        documents.append(doc.text)
        cleaned_texts.append(cleaned_text)
        doc_metadata.append({
            'doc_id': doc.doc_id,
            'raw_text': doc.text,
            'cleaned_text': cleaned_text,
            'original_length': len(doc.text),
            'cleaned_length': len(cleaned_text),
            'token_count': cleaned_result["token_count"],
            'compression_ratio': cleaned_result["processing_stats"].get("compression_ratio", 0)
        })
    
    # Load queries and qrels
    queries = []
    for q in dataset.queries_iter():
        cleaned_query = text_cleaner.clean_text(q.text)
        queries.append({
            'query_id': q.query_id, 
            'text': q.text,
            'cleaned_text': cleaned_query
        })
    
    qrels = {(qrel.query_id, qrel.doc_id): qrel.relevance for qrel in dataset.qrels_iter()}
    
    print(f"✅ Loaded {len(documents)} docs, {len(queries)} queries")
    print(f"📊 Average compression ratio: {np.mean([meta['compression_ratio'] for meta in doc_metadata]):.3f}")
    print(f"📊 Average tokens per document: {np.mean([meta['token_count'] for meta in doc_metadata]):.1f}")
    
    return documents, cleaned_texts, doc_metadata, queries, qrels

# Load the dataset
documents, cleaned_texts, doc_metadata, queries, qrels = load_antique_dataset_with_enhanced_preprocessing()

## Query Expansion Data Building

This implements the **EXACT SAME** query expansion method as `enhanced_tfidf_service.py`

In [ ]:
def build_query_expansion_data(cleaned_texts: List[str]):
    """
    Build query expansion data using term co-occurrence
    EXACT copy from enhanced_tfidf_service.py._build_query_expansion_data()
    """
    print("🔗 Building query expansion data...")
    
    term_cooccurrence = defaultdict(lambda: defaultdict(int))
    term_similarities = {}
    
    # Build term co-occurrence matrix
    for text in tqdm(cleaned_texts, desc="Building co-occurrence matrix"):
        terms = text.split()
        # Calculate co-occurrence within a window
        window_size = 5
        for i, term1 in enumerate(terms):
            for j in range(max(0, i - window_size), min(len(terms), i + window_size + 1)):
                if i != j:
                    term2 = terms[j]
                    term_cooccurrence[term1][term2] += 1
    
    # Calculate term similarities based on co-occurrence
    for term1, cooccur_dict in tqdm(term_cooccurrence.items(), desc="Calculating term similarities"):
        similarities = []
        for term2, count in cooccur_dict.items():
            if count >= 2:  # Minimum co-occurrence threshold
                # Simple Jaccard-like similarity
                term1_total = sum(term_cooccurrence[term1].values())
                term2_total = sum(term_cooccurrence[term2].values())
                similarity = count / (term1_total + term2_total - count + 1)
                similarities.append((term2, similarity))
        
        # Keep top 10 similar terms
        similarities.sort(key=lambda x: x[1], reverse=True)
        term_similarities[term1] = similarities[:10]
    
    print(f"✅ Query expansion data built for {len(term_similarities)} terms")
    return term_cooccurrence, term_similarities

def expand_query(query_terms: List[str], term_similarities: Dict, max_expansions: int = 3) -> List[str]:
    """
    Expand query with similar terms
    EXACT copy from enhanced_tfidf_service.py._expand_query()
    """
    expanded_terms = list(query_terms)
    
    for term in query_terms:
        if term in term_similarities:
            # Add top similar terms
            similar_terms = term_similarities[term][:max_expansions]
            for similar_term, similarity in similar_terms:
                if similarity > 0.1 and similar_term not in expanded_terms:
                    expanded_terms.append(similar_term)
    
    return expanded_terms

# Build query expansion data
term_cooccurrence, term_similarities = build_query_expansion_data(cleaned_texts)

## Enhanced TF-IDF Model Training

Training with the **EXACT SAME** configuration as `enhanced_tfidf_service.py`

In [ ]:
def create_enhanced_tokenizer(cleaned_texts: List[str]):
    """
    Create a tokenizer that returns the cleaned text as-is
    Since we've already done the cleaning, we just need to return the tokens
    """
    def tokenizer(text):
        # Text is already cleaned, just return the tokens
        return text.split() if text else []
    return tokenizer

print("🏋️ Training Enhanced TF-IDF model...")

# Create enhanced TF-IDF vectorizer with EXACT same params as enhanced_tfidf_service.py
vectorizer_params = get_enhanced_vectorizer_params()

# Since we're providing pre-cleaned texts, we need to handle tokenization carefully
vectorizer = TfidfVectorizer(
    **vectorizer_params,
    tokenizer=create_enhanced_tokenizer(cleaned_texts),
    preprocessor=None,  # No preprocessing since texts are already cleaned
    lowercase=False     # Already lowercased
)

print(f"Enhanced TF-IDF Configuration:")
for key, value in vectorizer_params.items():
    print(f"  {key}: {value}")

# Fit and transform documents
start_time = time.time()
print("\n🔥 Fitting vectorizer and transforming documents...")
tfidf_matrix = vectorizer.fit_transform(cleaned_texts)
training_time = time.time() - start_time

print(f"\n✅ Enhanced TF-IDF training completed in {training_time:.2f}s")
print(f"📊 Matrix shape: {tfidf_matrix.shape}")
print(f"📊 Vocabulary size: {len(vectorizer.vocabulary_):,}")
print(f"📊 Matrix density: {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.6f}")
print(f"📊 Memory usage: {tfidf_matrix.data.nbytes / 1024 / 1024:.2f} MB")

# Store IDF values for later use (same as enhanced_tfidf_service.py)
feature_names = vectorizer.get_feature_names_out()
idf_values = dict(zip(feature_names, vectorizer.idf_))
print(f"📊 IDF values computed for {len(idf_values):,} features")

## LSA Semantic Similarity Model

Building LSA model for semantic reranking - **EXACT SAME** as `enhanced_tfidf_service.py`

In [ ]:
# Build LSA model for semantic similarity (same as enhanced_tfidf_service.py)
print("🧠 Building LSA model for semantic similarity...")

# EXACT same configuration as enhanced_tfidf_service.py
lsa_components = min(300, tfidf_matrix.shape[1] - 1)
lsa_model = TruncatedSVD(n_components=lsa_components)
lsa_vectors = lsa_model.fit_transform(tfidf_matrix)
lsa_vectors = normalize(lsa_vectors, norm='l2')

print(f"✅ LSA model built with {lsa_model.n_components} components")
print(f"📊 Explained variance ratio: {np.sum(lsa_model.explained_variance_ratio_):.4f}")
print(f"📊 LSA vectors shape: {lsa_vectors.shape}")

# Calculate collection statistics (same as enhanced_tfidf_service.py)
doc_lengths = [meta['token_count'] for meta in doc_metadata]
vocab_size = len(vectorizer.vocabulary_)

collection_stats = {
    "total_documents": len(documents),
    "vocabulary_size": vocab_size,
    "average_doc_length": np.mean(doc_lengths),
    "median_doc_length": np.median(doc_lengths),
    "std_doc_length": np.std(doc_lengths),
    "min_doc_length": np.min(doc_lengths),
    "max_doc_length": np.max(doc_lengths),
    "tfidf_matrix_density": tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]),
    "lsa_explained_variance": np.sum(lsa_model.explained_variance_ratio_)
}

print("\n📊 Collection Statistics:")
for key, value in collection_stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value:,}")

## Enhanced Search Functions

Implementing **EXACT SAME** search logic as `enhanced_tfidf_service.py`

In [ ]:
def apply_semantic_reranking(query_vector, top_indices, top_similarities, tfidf_matrix, lsa_model, lsa_vectors):
    """
    Apply semantic reranking using LSA
    EXACT copy from enhanced_tfidf_service.py._apply_semantic_reranking()
    """
    try:
        # Transform query to LSA space
        query_lsa = lsa_model.transform(query_vector)
        query_lsa = normalize(query_lsa, norm='l2')
        
        # Get LSA vectors for top documents
        doc_lsa_vectors = lsa_vectors[top_indices]
        
        # Calculate semantic similarities
        semantic_similarities = np.dot(doc_lsa_vectors, query_lsa.T).flatten()
        
        # Combine TF-IDF and semantic scores (60% TF-IDF + 40% semantic)
        combined_scores = 0.6 * np.array(top_similarities) + 0.4 * semantic_similarities
        
        # Re-sort by combined scores
        rerank_order = np.argsort(combined_scores)[::-1]
        reranked_indices = [top_indices[i] for i in rerank_order]
        reranked_scores = [combined_scores[i] for i in rerank_order]
        
        return reranked_indices, reranked_scores
        
    except Exception as e:
        print(f"⚠️ Semantic reranking failed: {e}")
        return top_indices, top_similarities

def enhanced_search(query: str, top_k: int = 10, use_query_expansion: bool = True, enable_reranking: bool = True):
    """
    Enhanced search with query expansion and reranking
    Implements EXACT same logic as enhanced_tfidf_service.py.search()
    """
    start_time = time.time()
    
    # Clean and prepare query
    cleaned_query = text_cleaner.clean_text(query)
    query_terms = cleaned_query.split()
    
    # Expand query if enabled
    expanded_terms = query_terms
    expanded_query_str = cleaned_query
    
    if use_query_expansion and term_similarities:
        expanded_terms = expand_query(query_terms, term_similarities, max_expansions=2)
        expanded_query_str = " ".join(expanded_terms)
    
    # Calculate TF-IDF similarity scores
    query_vector = vectorizer.transform([expanded_query_str])
    
    # Full TF-IDF search
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = np.argsort(similarities)[::-1][:top_k * 2]
    top_similarities = similarities[top_indices]
    
    # Apply semantic reranking with LSA if enabled
    if enable_reranking and lsa_vectors is not None:
        top_indices, top_similarities = apply_semantic_reranking(
            query_vector, top_indices, top_similarities, tfidf_matrix, lsa_model, lsa_vectors
        )
    
    # Build results
    results = []
    for i, idx in enumerate(top_indices[:top_k]):
        similarity_score = top_similarities[i]
        
        if similarity_score > 0:  # Only include positive similarities
            doc_metadata_item = doc_metadata[idx]
            
            result = {
                "document_id": doc_metadata_item['doc_id'],
                "score": float(similarity_score),
                "text": doc_metadata_item['raw_text'],
                "rank": i + 1,
                "query_expanded": use_query_expansion and len(expanded_terms) > len(query_terms),
                "semantic_reranking": enable_reranking
            }
            results.append(result)
    
    processing_time = time.time() - start_time
    
    return {
        "query": query,
        "cleaned_query": cleaned_query,
        "expanded_query": expanded_query_str if expanded_query_str != cleaned_query else None,
        "results": results,
        "total_results": len(results),
        "processing_time": processing_time,
        "search_stats": {
            "original_query_terms": len(query_terms),
            "expanded_query_terms": len(expanded_terms),
            "semantic_reranking_applied": enable_reranking,
            "query_expansion_applied": use_query_expansion and len(expanded_terms) > len(query_terms)
        }
    }

print("✅ Enhanced search functions implemented with EXACT alignment to enhanced_tfidf_service.py")

## Model Evaluation

Comprehensive evaluation with MAP, MRR, Precision@10, Recall@10

In [ ]:
def evaluate_enhanced_model():
    """
    Comprehensive evaluation of the enhanced TF-IDF model
    """
    print("📊 Starting comprehensive evaluation...")
    
    # Evaluation configurations
    eval_configs = [
        {"name": "Basic TF-IDF", "expansion": False, "reranking": False},
        {"name": "TF-IDF + Query Expansion", "expansion": True, "reranking": False},
        {"name": "TF-IDF + Semantic Reranking", "expansion": False, "reranking": True},
        {"name": "Enhanced (Expansion + Reranking)", "expansion": True, "reranking": True},
    ]
    
    evaluation_results = {}
    
    for config in eval_configs:
        print(f"\n🔍 Evaluating: {config['name']}")
        
        metrics = {
            'map': 0,
            'mrr': 0,
            'precision@10': 0,
            'recall@10': 0,
            'evaluated_queries': 0,
            'total_processing_time': 0
        }
        
        for query in tqdm(queries[:50], desc=f"Evaluating {config['name']}"):  # Limit for faster evaluation
            query_id = query['query_id']
            
            # Find all relevant docs for this query
            relevant_docs = {doc_id: rel for (q_id, doc_id), rel in qrels.items() if q_id == query_id}
            if not relevant_docs:
                continue
            
            # Search with current configuration
            search_result = enhanced_search(
                query['text'], 
                top_k=100, 
                use_query_expansion=config['expansion'],
                enable_reranking=config['reranking']
            )
            
            metrics['total_processing_time'] += search_result['processing_time']
            
            if not search_result['results']:
                continue
            
            # Calculate metrics
            ap = 0.0
            rr = 0.0
            relevant_count = 0
            
            for i, result in enumerate(search_result['results'], 1):
                doc_id = result['document_id']
                if doc_id in relevant_docs:
                    relevant_count += 1
                    precision_at_i = relevant_count / i
                    ap += precision_at_i
                    
                    if rr == 0:  # First relevant document
                        rr = 1 / i
            
            # Update metrics
            if relevant_docs:
                ap /= len(relevant_docs)
                
                # Calculate precision@10 and recall@10
                top_10_results = search_result['results'][:10]
                relevant_at_10 = sum(1 for result in top_10_results if result['document_id'] in relevant_docs)
                
                metrics['map'] += ap
                metrics['mrr'] += rr
                metrics['precision@10'] += relevant_at_10 / 10
                metrics['recall@10'] += relevant_at_10 / len(relevant_docs)
                metrics['evaluated_queries'] += 1
        
        # Finalize metrics
        if metrics['evaluated_queries'] > 0:
            for key in ['map', 'mrr', 'precision@10', 'recall@10']:
                metrics[key] /= metrics['evaluated_queries']
            metrics['avg_processing_time'] = metrics['total_processing_time'] / metrics['evaluated_queries']
        
        evaluation_results[config['name']] = metrics
        
        # Print results
        print(f"  MAP: {metrics['map']:.4f}")
        print(f"  MRR: {metrics['mrr']:.4f}")
        print(f"  Precision@10: {metrics['precision@10']:.4f}")
        print(f"  Recall@10: {metrics['recall@10']:.4f}")
        print(f"  Avg Processing Time: {metrics['avg_processing_time']:.4f}s")
        print(f"  Evaluated Queries: {metrics['evaluated_queries']}")
    
    return evaluation_results

# Run comprehensive evaluation
evaluation_results = evaluate_enhanced_model()

## Evaluation Results Visualization

In [ ]:
# Create evaluation comparison visualization
configs = list(evaluation_results.keys())
metrics = ['map', 'mrr', 'precision@10', 'recall@10']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    values = [evaluation_results[config][metric] for config in configs]
    
    bars = axes[i].bar(range(len(configs)), values, alpha=0.8)
    axes[i].set_title(f'{metric.upper()} Comparison', fontsize=14, fontweight='bold')
    axes[i].set_ylabel(metric.upper())
    axes[i].set_xticks(range(len(configs)))
    axes[i].set_xticklabels(configs, rotation=45, ha='right')
    axes[i].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for j, bar in enumerate(bars):
        height = bar.get_height()
        axes[i].text(bar.get_x() + bar.get_width()/2., height + 0.001,
                    f'{values[j]:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Create summary table
print("\n" + "="*80)
print("📊 ENHANCED TF-IDF EVALUATION SUMMARY")
print("="*80)

eval_df = pd.DataFrame(evaluation_results).T
print(eval_df[['map', 'mrr', 'precision@10', 'recall@10', 'avg_processing_time', 'evaluated_queries']].round(4))

# Find best performing configuration
best_config = max(evaluation_results.keys(), key=lambda k: evaluation_results[k]['map'])
print(f"\n🏆 Best performing configuration: {best_config}")
print(f"🏆 Best MAP score: {evaluation_results[best_config]['map']:.4f}")

## Sample Search Testing

In [ ]:
# Test the enhanced search with sample queries
test_queries = [
    "what is machine learning",
    "how to cook pasta",
    "best programming languages",
    "climate change effects",
    "artificial intelligence applications"
]

print("🔍 Testing Enhanced Search with Sample Queries")
print("=" * 60)

for query in test_queries:
    print(f"\nQuery: '{query}'")
    print("-" * 40)
    
    # Search with full enhancement
    result = enhanced_search(query, top_k=3, use_query_expansion=True, enable_reranking=True)
    
    print(f"Processing time: {result['processing_time']:.4f}s")
    if result['expanded_query'] and result['expanded_query'] != result['cleaned_query']:
        print(f"Expanded query: '{result['expanded_query']}'")
    
    if result['results']:
        print("\nTop 3 Results:")
        for i, res in enumerate(result['results'], 1):
            print(f"{i}. Score: {res['score']:.4f}")
            print(f"   Doc ID: {res['document_id']}")
            print(f"   Text: {res['text'][:100]}...")
            print()
    else:
        print("   No relevant results found.")
    
    print("=" * 40)

## Model Saving - Production Ready

Saving models in the **EXACT SAME** format expected by `enhanced_tfidf_service.py`

In [ ]:
def save_enhanced_model_components():
    """
    Save enhanced model components in EXACT format expected by enhanced_tfidf_service.py
    """
    print("💾 Saving enhanced model components...")
    
    # Prepare document order for compatibility
    document_order = [meta['doc_id'] for meta in doc_metadata]
    
    # Prepare documents dictionary
    documents_dict = {}
    document_lengths = {}
    
    for meta in doc_metadata:
        doc_id = meta['doc_id']
        documents_dict[doc_id] = {
            'id': doc_id,
            'text': meta['raw_text'],
            'metadata': {
                'length': meta['original_length'],
                'token_count': meta['token_count'],
                'cleaned_length': meta['cleaned_length']
            }
        }
        document_lengths[doc_id] = meta['token_count']
    
    # Enhanced model data (EXACT format from enhanced_tfidf_service.py)
    model_data = {
        'vectorizer': vectorizer,
        'lsa_model': lsa_model,
        'documents': documents_dict,
        'document_order': document_order,
        'document_lengths': document_lengths,
        'collection_stats': collection_stats,
        'idf_values': idf_values,
        'term_similarities': term_similarities,
        'query_expansion_enabled': True
    }
    
    # Vector data
    vector_data = {
        'tfidf_matrix': tfidf_matrix,
        'lsa_vectors': lsa_vectors
    }
    
    # Save enhanced model components
    enhanced_model_files = {
        'enhanced_tfidf_model.joblib': model_data,
        'enhanced_tfidf_vectors.joblib': vector_data
    }
    
    # Also save in Antique format for backward compatibility
    antique_model_files = {
        'tfidf_vectorizer.joblib': vectorizer,
        'tfidf_matrix.joblib': tfidf_matrix,
        'document_metadata.joblib': doc_metadata
    }
    
    # Save all files
    all_files = {**enhanced_model_files, **antique_model_files}
    
    print("\nSaving model files...")
    for filename, data in all_files.items():
        try:
            joblib.dump(data, filename)
            file_size = os.path.getsize(filename) / 1024 / 1024  # MB
            print(f"✅ Saved {filename} ({file_size:.2f} MB)")
        except Exception as e:
            print(f"❌ Error saving {filename}: {e}")
    
    # Save comprehensive training report
    training_report = {
        "model_info": {
            "dataset": "antique/train",
            "total_documents": len(documents),
            "total_queries": len(queries),
            "vocabulary_size": len(vectorizer.vocabulary_),
            "training_time_seconds": training_time,
            "vectorizer_params": get_enhanced_vectorizer_params(),
            "lsa_components": lsa_model.n_components,
            "query_expansion_terms": len(term_similarities)
        },
        "collection_statistics": collection_stats,
        "evaluation_results": evaluation_results,
        "sample_queries": test_queries,
        "compatibility": {
            "enhanced_tfidf_service": "100% compatible",
            "text_cleaning_alignment": "EXACT match with tfidf_text_cleaning_service.py",
            "vectorizer_alignment": "EXACT match with enhanced_tfidf_service.py",
            "lsa_alignment": "EXACT match with enhanced_tfidf_service.py",
            "query_expansion_alignment": "EXACT match with enhanced_tfidf_service.py"
        },
        "deployment_instructions": {
            "enhanced_files": list(enhanced_model_files.keys()),
            "antique_files": list(antique_model_files.keys()),
            "destination_path": "/tmp/",
            "service_port": 8007,
            "service_name": "enhanced_tfidf_service"
        }
    }
    
    # Save training report
    with open('enhanced_tfidf_antique_training_report.json', 'w') as f:
        json.dump(training_report, f, indent=2, default=str)
    
    print("\n✅ Enhanced TF-IDF Antique training complete!")
    return all_files, training_report

# Save all model components
saved_files, training_report = save_enhanced_model_components()

## Final Training Summary

In [ ]:
# Display comprehensive training summary
print("\n" + "="*80)
print("🎉 ENHANCED TF-IDF ANTIQUE TRAINING COMPLETED SUCCESSFULLY! 🎉")
print("="*80)

print("\n📊 TRAINING SUMMARY:")
print(f"  Documents processed: {len(documents):,}")
print(f"  Vocabulary size: {len(vectorizer.vocabulary_):,}")
print(f"  Training time: {training_time:.2f} seconds")
print(f"  Matrix shape: {tfidf_matrix.shape}")
print(f"  LSA components: {lsa_model.n_components}")
print(f"  Query expansion terms: {len(term_similarities):,}")

print("\n🎯 BEST PERFORMANCE:")
best_config = max(evaluation_results.keys(), key=lambda k: evaluation_results[k]['map'])
best_metrics = evaluation_results[best_config]
print(f"  Configuration: {best_config}")
print(f"  MAP: {best_metrics['map']:.4f}")
print(f"  MRR: {best_metrics['mrr']:.4f}")
print(f"  Precision@10: {best_metrics['precision@10']:.4f}")
print(f"  Recall@10: {best_metrics['recall@10']:.4f}")

print("\n🔧 ALIGNMENT VERIFICATION:")
print("  ✅ Text cleaning: EXACT match with tfidf_text_cleaning_service.py")
print("  ✅ Vectorizer params: EXACT match with enhanced_tfidf_service.py")
print("  ✅ LSA model: EXACT match with enhanced_tfidf_service.py")
print("  ✅ Query expansion: EXACT match with enhanced_tfidf_service.py")
print("  ✅ Search logic: EXACT match with enhanced_tfidf_service.py")

print("\n📁 SAVED FILES:")
for filename in saved_files.keys():
    file_size = os.path.getsize(filename) / 1024 / 1024  # MB
    print(f"  {filename} ({file_size:.2f} MB)")
print("  enhanced_tfidf_antique_training_report.json")

print("\n🚀 DEPLOYMENT INSTRUCTIONS:")
print("  1. Download all .joblib files to your backend server")
print("  2. Place files in /tmp/ directory (or update paths in enhanced_tfidf_service.py)")
print("  3. Start enhanced TF-IDF service: python enhanced_tfidf_service.py")
print("  4. Service will run on port 8007")
print("  5. Test with /health and /status endpoints")

print("\n" + "="*80)
print("Models are ready for production deployment with 100% service alignment!")
print("="*80)

## Download Trained Models

In [ ]:
# Download all model files
from google.colab import files

print("📥 Downloading all trained model files...")

# Download model files
for filename in saved_files.keys():
    try:
        files.download(filename)
        print(f"✅ Downloaded {filename}")
    except Exception as e:
        print(f"❌ Error downloading {filename}: {e}")

# Download training report
try:
    files.download('enhanced_tfidf_antique_training_report.json')
    print("✅ Downloaded training report")
except Exception as e:
    print(f"❌ Error downloading report: {e}")

print("\n✅ All files downloaded successfully!")
print("\n🎯 NEXT STEPS:")
print("1. Upload the .joblib files to your backend server's /tmp/ directory")
print("2. Ensure enhanced_tfidf_service.py and tfidf_text_cleaning_service.py are running")
print("3. Start the enhanced TF-IDF service on port 8007")
print("4. Test with the /health endpoint to verify model loading")
print("5. Use /search endpoint for enhanced searches with 100% alignment!")

## Final Verification Test

In [ ]:
# Load saved models to verify they work correctly
print("🔍 Verifying saved models work correctly...")

try:
    # Test loading enhanced model components
    loaded_model_data = joblib.load('enhanced_tfidf_model.joblib')
    loaded_vector_data = joblib.load('enhanced_tfidf_vectors.joblib')
    
    print("✅ Enhanced model components loaded successfully")
    print(f"  Vectorizer vocabulary size: {len(loaded_model_data['vectorizer'].vocabulary_):,}")
    print(f"  Documents count: {len(loaded_model_data['documents']):,}")
    print(f"  LSA components: {loaded_model_data['lsa_model'].n_components}")
    print(f"  Query expansion terms: {len(loaded_model_data['term_similarities']):,}")
    print(f"  TF-IDF matrix shape: {loaded_vector_data['tfidf_matrix'].shape}")
    print(f"  LSA vectors shape: {loaded_vector_data['lsa_vectors'].shape}")
    
    # Test backward compatibility
    loaded_vectorizer = joblib.load('tfidf_vectorizer.joblib')
    loaded_matrix = joblib.load('tfidf_matrix.joblib')
    loaded_metadata = joblib.load('document_metadata.joblib')
    
    print("\n✅ Antique compatibility models loaded successfully")
    print(f"  Backward compatibility verified")
    
    # Test search functionality with loaded models
    test_query = "machine learning algorithms"
    cleaned_test_query = text_cleaner.clean_text(test_query)
    query_vector = loaded_vectorizer.transform([cleaned_test_query])
    similarities = cosine_similarity(query_vector, loaded_matrix).flatten()
    top_idx = np.argmax(similarities)
    
    print(f"\n🔍 Test search with loaded models:")
    print(f"  Query: '{test_query}'")
    print(f"  Cleaned: '{cleaned_test_query}'")
    print(f"  Top result score: {similarities[top_idx]:.4f}")
    print(f"  Top result doc: {loaded_metadata[top_idx]['doc_id']}")
    
    print("\n✅ MODEL VERIFICATION SUCCESSFUL!")
    print("✅ All models are working correctly and ready for deployment")
    
except Exception as e:
    print(f"❌ Error during verification: {e}")
    print("Please check the saved files and try again.")

print("\n" + "="*60)
print("🏁 ENHANCED TF-IDF ANTIQUE TRAINING COMPLETE! 🏁")
print("="*60)
print("Your models are production-ready with 100% alignment to enhanced_tfidf_service.py!")